In [1]:
import numpy as np
import cv2
import os
import torch as t
import visdom
import time

from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils import data

vis = visdom.Visdom(env = 'grad_FC_txt')


In [2]:
kernel_ = np.array([[1/16,1/8,1/16],[1/8,1/4,1/8],[1/16,1/8,1/16]])

p = np.zeros((28,3,3))
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[0,1] = 1; p[0] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[0,2] = 1; p[1] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[1,0] = 1; p[2] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[1,2] = 1; p[3] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,0] = 1; p[4] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,1] = 1; p[5] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,2] = 1; p[6] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[0,2] = 1; p[7] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[1,0] = 1; p[8] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[1,2] = 1; p[9] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,0] = 1; p[10] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,1] = 1; p[11] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,2] = 1; p[12] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[1,0] = 1; p[13] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[1,2] = 1; p[14] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,0] = 1; p[15] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,1] = 1; p[16] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,2] = 1; p[17] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[1,2] = 1; p[18] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,0] = 1; p[19] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,1] = 1; p[20] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,2] = 1; p[21] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,0] = 1; p[22] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,1] = 1; p[23] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,2] = 1; p[24] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,0] = 1;zeros[2,1] = 1; p[25] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,0] = 1;zeros[2,2] = 1; p[26] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,1] = 1;zeros[2,2] = 1; p[27] = zeros


In [4]:
img_2grad = np.zeros((28,30,30,3))
var_array = np.zeros((84))
var_array1 = np.zeros((28))
aver_array = np.zeros((84))
aver_array1 = np.zeros((28))
var_array_max_all = 0
var_array_min_all = 100000
aver_array_max_all = 0
aver_array_min_all = 100000
var_array_max1_all = 0
var_array_min1_all = 100000
aver_array_max1_all = 0
aver_array_min1_all = 100000
dir_path = "D:\\newfolder43_1"
save_path = "C:\\newfolder43_1_txt2"
dirs = os.listdir(dir_path)
for i in range(len(dirs)):
    pic_names = os.listdir(dir_path+"\\"+dirs[i])
    for n in range(len(pic_names)):
        pic_path = dir_path+"\\"+dirs[i]+"\\"+pic_names[n]
        print(pic_path)
        #pic_path_gbk = pic_path.encode("gbk")
        #pic_path_ascii = pic_path_gbk.decode()
        #image = cv2.imread(pic_path_ascii)
        image=cv2.imdecode(np.fromfile(pic_path,dtype=np.uint8),-1)
        all_result_array = np.array([])
        all_var_array = np.array([])
        all_aver_array = np.array([])
        all_var_array1 = np.array([])
        all_aver_array1 = np.array([])
        p=6
        for hor_num in range(p):
            for col_num in range(p):
                if hor_num < p-1 and col_num <p-1:
                    image1 = image[(20*hor_num):(20*(hor_num)+30),(20*col_num):(20*(col_num)+30)]
                else:
                    image1 = image[(20*hor_num):120,(20*col_num):120]
                #image1 = image1/255.0
                image_gaussion = cv2.filter2D(image1,cv2.CV_16S,kernel_)
                #image_gaussion = np.true_divide(image_gaussion,255.0)
                for m in range(28):
                    img_2grad[m] = (cv2.filter2D(image_gaussion,cv2.CV_16S,p[m]))/255.0
                    
                    var_array[3*m] = img_2grad[m][:,:,0].std()
                    var_array[3*m+1] = img_2grad[m][:,:,1].std()
                    var_array[3*m+2] = img_2grad[m][:,:,2].std()
                    #aver_array[3*m] = img_2grad[m][:,:,0].mean()
                    #aver_array[3*m+1] = img_2grad[m][:,:,1].mean()
                    #aver_array[3*m+2] = img_2grad[m][:,:,2].mean()
                    #var_array1[m] = img_2grad[m].std()
                    #aver_array1[m] = img_2grad[m].mean()
                
                all_var_array = np.hstack((all_var_array,var_array))
                #all_aver_array = np.hstack((all_aver_array,aver_array))   
                #all_var_array1 = np.hstack((all_var_array1,var_array1))
                #all_aver_array1 = np.hstack((all_aver_array1,aver_array1))
                
        
        
        #result_array = np.hstack((all_var_array,all_var_array))
        name = save_path+"\\"+dirs[i]+"\\"+(pic_names[n].split("."))[0]+".txt"
        fw = open(name,"w")
        fw.write(','.join(str(num) for num in all_var_array))
        fw.close()


D:\newfolder43_1\clear_face\picture100_147_.png


TypeError: 'int' object is not subscriptable

In [20]:
fr = open("C:\\newfolder43_1_txt2\\clear_face\\picture_3144_.txt")
data_str = fr.read()
array = np.array(data_str.split(",")).astype(float)
fr.close()
input_numbers = array.shape[0]
input_numbers

2688

In [21]:
#网络model
class FC_Net(nn.Module):
    def __init__(self,input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim):
        super(FC_Net,self).__init__()
        self.layer1 = nn.Sequential(
                                    
                                    nn.Linear(input_dim,n_hidden_1),
                                    nn.BatchNorm1d(n_hidden_1),
                                    nn.ReLU(True)
                                    )
        self.layer2 = nn.Sequential(
                                    nn.Linear(n_hidden_1,n_hidden_2),
                                    nn.BatchNorm1d(n_hidden_2),
                                    nn.ReLU(True)
                                    )
        self.layer3 = nn.Sequential(
                                    nn.Linear(n_hidden_2,n_hidden_3),
                                    nn.BatchNorm1d(n_hidden_3),
                                    nn.ReLU(True)
                                    )
        self.layer_out = nn.Sequential(
                                    nn.Linear(n_hidden_3,output_dim),
                                    #nn.BatchNorm1d(output_dim),
                                    #nn.ReLU(True)
                                       )

    def forward(self,x):
        #print("x.shape",x.shape)
        x = self.layer1(x)
        #print("x.shape1",x.shape)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer_out(x)
        #x = self.softmax(x)
        #print(x)
        return x

In [22]:
class txt_dataset(data.Dataset):
    def __init__(self,root,transforms0=None):
        self.paths = [root+"\\"+path+"\\"+val for path in os.listdir(root) for val in os.listdir(root+"\\" +path)]
        self.transforms0 = transforms0
        
    def __getitem__(self,index):   
        data_path = self.paths[index]
        label = 1 if 'clear_face' in data_path.split("\\")[-2] else 0
        fr = open(data_path)
        data_str = fr.read()
        array = np.array(data_str.split(",")).astype(float)
        fr.close()
        data = t.from_numpy(array)        
        
        return data,label
    
    def __len__(self):
        return len(self.paths)
                                        

In [23]:
batch_size = 64
learning_rate = 1e-4
num_epoches = 500
weight_decay = 1e-4

In [24]:
dataset0 = txt_dataset("C:\\newfolder43_1_txt2")
dataset0[0][1]


1

In [25]:
dataloader = DataLoader(dataset0,batch_size=batch_size,shuffle=True)
#dataiter = iter(dataloader)
#imgs,labels = next(dataiter)
#labels

In [26]:
input_dim=2688
n_hidden_1=128
n_hidden_2=64
n_hidden_3=32
output_dim=2
model = FC_Net(input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim)

In [27]:
if t.cuda.is_available():
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()
criterion.cuda() 
optimizer = optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay) 

In [ ]:
print("training start time  <===>  ",time.localtime())
sum_loss = t.Tensor([0])
for epoch in range(0,num_epoches):
    for i,data in enumerate(dataloader,0):
        data_batch, label = data        
        data_batch, label = Variable(data_batch.float()).cuda(),Variable(label).cuda()
        output = model(data_batch)
        optimizer.zero_grad()
        loss = criterion(output,label)
        loss.backward() 
        optimizer.step()
        
        sum_loss = t.cat((sum_loss,loss.cpu().data),0)
        x = i+epoch*len(dataloader)
        if x%50 == 49:
            loss_aver = sum_loss.mean()
            sum_loss = t.Tensor([0])
            vis.line(X=t.Tensor([x]), Y=t.Tensor([loss_aver]), win='blur_new1',opts={'title': 'loss-batch_new1', 'xlabel':'batch_num','ylabel':'loss_aver'},update='append' if x>49 else None)
            #vis.updateTrace(X=t.Tensor([x]),Y=t.Tensor([loss_aver]),win='face_angle', name='7')
t.save(model.state_dict(),"C:\\Users\\SiChen\\face_sys\\grad_var_full_layers\\new_model_save\\FC_500_128_64_NoSoftmax_4x4x28x8std_mean_NEWguiyihua_txt_lr1e-4.pth") 
print("training start time  <===>  ",time.localtime())

training start time  <===>   time.struct_time(tm_year=2018, tm_mon=7, tm_mday=1, tm_hour=16, tm_min=40, tm_sec=24, tm_wday=6, tm_yday=182, tm_isdst=0)


In [13]:
t.save(model.state_dict(),"C:\\Users\\SiChen\\face_sys\\grad_var_full_layers\\new_model_save\\FC_500_128_64_NoSoftmax_4x4x28x4std_NEWguiyihua_txt_lr1e-4.pth")

In [15]:
from torch.utils import data

In [14]:
img_2grad = np.zeros((28,30,30,3))
var_array = np.zeros((84))
var_array1 = np.zeros((28))
aver_array = np.zeros((84))
aver_array1 = np.zeros((28))
var_array_max_all = 0
var_array_min_all = 100000
aver_array_max_all = 0
aver_array_min_all = 100000
var_array_max1_all = 0
var_array_min1_all = 100000
aver_array_max1_all = 0
aver_array_min1_all = 100000
dir_path = "D:\\newfolder43_2"
save_path = "D:\\newfolder43_2_txt"
dirs = os.listdir(dir_path)
for i in range(len(dirs)):
    pic_names = os.listdir(dir_path+"\\"+dirs[i])
    for n in range(len(pic_names)):
        pic_path = dir_path+"\\"+dirs[i]+"\\"+pic_names[n]
        print(pic_path)
        #pic_path_gbk = pic_path.encode("gbk")
        #pic_path_ascii = pic_path_gbk.decode()
        #image = cv2.imread(pic_path_ascii)
        image=cv2.imdecode(np.fromfile(pic_path,dtype=np.uint8),-1)
        all_result_array = np.array([])
        all_var_array = np.array([])
        all_aver_array = np.array([])
        all_var_array1 = np.array([])
        all_aver_array1 = np.array([])
        for hor_num in range(4):
            for col_num in range(4):
                image1 = image[(30*hor_num):(30*(hor_num+1)),(30*col_num):(30*(col_num+1))]
                #image1 = image1/255.0
                image_gaussion = cv2.filter2D(image1,cv2.CV_16S,kernel_)
                #image_gaussion = np.true_divide(image_gaussion,255.0)
                for m in range(28):
                    img_2grad[m] = (cv2.filter2D(image_gaussion,cv2.CV_16S,p[m]))/255.0
                    
                    var_array[3*m] = img_2grad[m][:,:,0].std()
                    var_array[3*m+1] = img_2grad[m][:,:,1].std()
                    var_array[3*m+2] = img_2grad[m][:,:,2].std()
                    #aver_array[3*m] = img_2grad[m][:,:,0].mean()
                    #aver_array[3*m+1] = img_2grad[m][:,:,1].mean()
                    #aver_array[3*m+2] = img_2grad[m][:,:,2].mean()
                    var_array1[m] = img_2grad[m].std()
                    #aver_array1[m] = img_2grad[m].mean()
                
                all_var_array = np.hstack((all_var_array,var_array))
                #all_aver_array = np.hstack((all_aver_array,aver_array))   
                all_var_array1 = np.hstack((all_var_array1,var_array1))
                #all_aver_array1 = np.hstack((all_aver_array1,aver_array1))
                
        
        
        #result_array = np.hstack((all_var_array,all_aver_array,all_var_array1,all_aver_array1))
        result_array = np.hstack((all_var_array,all_var_array1))
        name = save_path+"\\"+dirs[i]+"\\"+(pic_names[n].split("."))[0]+".txt"
        fw = open(name,"w")
        fw.write(','.join(str(num) for num in result_array))
        fw.close()


D:\newfolder43_2\clear_face\picture_1000_.png
D:\newfolder43_2\clear_face\picture_1002_.png
D:\newfolder43_2\clear_face\picture_1004_.png
D:\newfolder43_2\clear_face\picture_1005_.png
D:\newfolder43_2\clear_face\picture_1008_.png
D:\newfolder43_2\clear_face\picture_1009_.png
D:\newfolder43_2\clear_face\picture_100_.png
D:\newfolder43_2\clear_face\picture_1010_.png
D:\newfolder43_2\clear_face\picture_1013_.png
D:\newfolder43_2\clear_face\picture_1014_.png
D:\newfolder43_2\clear_face\picture_1015_.png
D:\newfolder43_2\clear_face\picture_1016_.png
D:\newfolder43_2\clear_face\picture_1018_.png
D:\newfolder43_2\clear_face\picture_1019_.png
D:\newfolder43_2\clear_face\picture_101_.png
D:\newfolder43_2\clear_face\picture_1020_.png
D:\newfolder43_2\clear_face\picture_1021_.png
D:\newfolder43_2\clear_face\picture_1024_.png
D:\newfolder43_2\clear_face\picture_1025_.png
D:\newfolder43_2\clear_face\picture_1027_.png
D:\newfolder43_2\clear_face\picture_1028_.png
D:\newfolder43_2\clear_face\picture_

D:\newfolder43_2\clear_face\picture_1322_.png
D:\newfolder43_2\clear_face\picture_1326_.png
D:\newfolder43_2\clear_face\picture_1327_.png
D:\newfolder43_2\clear_face\picture_1328_.png
D:\newfolder43_2\clear_face\picture_1329_.png
D:\newfolder43_2\clear_face\picture_132_.png
D:\newfolder43_2\clear_face\picture_1330_.png
D:\newfolder43_2\clear_face\picture_1331_.png
D:\newfolder43_2\clear_face\picture_1332_.png
D:\newfolder43_2\clear_face\picture_1333_.png
D:\newfolder43_2\clear_face\picture_1334_.png
D:\newfolder43_2\clear_face\picture_1335_.png
D:\newfolder43_2\clear_face\picture_1337_.png
D:\newfolder43_2\clear_face\picture_1338_.png
D:\newfolder43_2\clear_face\picture_1339_.png
D:\newfolder43_2\clear_face\picture_1340_.png
D:\newfolder43_2\clear_face\picture_1341_.png
D:\newfolder43_2\clear_face\picture_1342_.png
D:\newfolder43_2\clear_face\picture_1343_.png
D:\newfolder43_2\clear_face\picture_1344_.png
D:\newfolder43_2\clear_face\picture_1345_.png
D:\newfolder43_2\clear_face\picture

D:\newfolder43_2\clear_face\picture_1659_.png
D:\newfolder43_2\clear_face\picture_165_.png
D:\newfolder43_2\clear_face\picture_1661_.png
D:\newfolder43_2\clear_face\picture_1663_.png
D:\newfolder43_2\clear_face\picture_1664_.png
D:\newfolder43_2\clear_face\picture_1666_.png
D:\newfolder43_2\clear_face\picture_1668_.png
D:\newfolder43_2\clear_face\picture_166_.png
D:\newfolder43_2\clear_face\picture_1671_.png
D:\newfolder43_2\clear_face\picture_1673_.png
D:\newfolder43_2\clear_face\picture_1676_.png
D:\newfolder43_2\clear_face\picture_167_.png
D:\newfolder43_2\clear_face\picture_1683_.png
D:\newfolder43_2\clear_face\picture_1684_.png
D:\newfolder43_2\clear_face\picture_1685_.png
D:\newfolder43_2\clear_face\picture_1686_.png
D:\newfolder43_2\clear_face\picture_1688_.png
D:\newfolder43_2\clear_face\picture_168_.png
D:\newfolder43_2\clear_face\picture_1691_.png
D:\newfolder43_2\clear_face\picture_169_.png
D:\newfolder43_2\clear_face\picture_16_.png
D:\newfolder43_2\clear_face\picture_170_.

D:\newfolder43_2\clear_face\picture_2009_.png
D:\newfolder43_2\clear_face\picture_2014_.png
D:\newfolder43_2\clear_face\picture_201_.png
D:\newfolder43_2\clear_face\picture_2021_.png
D:\newfolder43_2\clear_face\picture_2028_.png
D:\newfolder43_2\clear_face\picture_202_.png
D:\newfolder43_2\clear_face\picture_203_.png
D:\newfolder43_2\clear_face\picture_2046_.png
D:\newfolder43_2\clear_face\picture_2051_.png
D:\newfolder43_2\clear_face\picture_2053_.png
D:\newfolder43_2\clear_face\picture_2059_.png
D:\newfolder43_2\clear_face\picture_2061_.png
D:\newfolder43_2\clear_face\picture_2062_.png
D:\newfolder43_2\clear_face\picture_2063_.png
D:\newfolder43_2\clear_face\picture_2064_.png
D:\newfolder43_2\clear_face\picture_2065_.png
D:\newfolder43_2\clear_face\picture_2066_.png
D:\newfolder43_2\clear_face\picture_2068_.png
D:\newfolder43_2\clear_face\picture_2077_.png
D:\newfolder43_2\clear_face\picture_2085_.png
D:\newfolder43_2\clear_face\picture_2088_.png
D:\newfolder43_2\clear_face\picture_2

D:\newfolder43_2\clear_face\picture_2519_.png
D:\newfolder43_2\clear_face\picture_2521_.png
D:\newfolder43_2\clear_face\picture_2525_.png
D:\newfolder43_2\clear_face\picture_2527_.png
D:\newfolder43_2\clear_face\picture_2529_.png
D:\newfolder43_2\clear_face\picture_252_.png
D:\newfolder43_2\clear_face\picture_2531_.png
D:\newfolder43_2\clear_face\picture_2533_.png
D:\newfolder43_2\clear_face\picture_2535_.png
D:\newfolder43_2\clear_face\picture_2537_.png
D:\newfolder43_2\clear_face\picture_2541_.png
D:\newfolder43_2\clear_face\picture_2547_.png
D:\newfolder43_2\clear_face\picture_2549_.png
D:\newfolder43_2\clear_face\picture_2550_.png
D:\newfolder43_2\clear_face\picture_2552_.png
D:\newfolder43_2\clear_face\picture_2559_.png
D:\newfolder43_2\clear_face\picture_2565_.png
D:\newfolder43_2\clear_face\picture_2578_.png
D:\newfolder43_2\clear_face\picture_2586_.png
D:\newfolder43_2\clear_face\picture_2587_.png
D:\newfolder43_2\clear_face\picture_2593_.png
D:\newfolder43_2\clear_face\picture

...

D:\newfolder43_2\fuzzy_face\picture_2211_.png
D:\newfolder43_2\fuzzy_face\picture_2215_.png
D:\newfolder43_2\fuzzy_face\picture_221_.png
D:\newfolder43_2\fuzzy_face\picture_2223_.png
D:\newfolder43_2\fuzzy_face\picture_2224_.png
D:\newfolder43_2\fuzzy_face\picture_2228_.png
D:\newfolder43_2\fuzzy_face\picture_222_.png
D:\newfolder43_2\fuzzy_face\picture_2230_.png
D:\newfolder43_2\fuzzy_face\picture_2236_.png
D:\newfolder43_2\fuzzy_face\picture_2238_.png
D:\newfolder43_2\fuzzy_face\picture_223_.png
D:\newfolder43_2\fuzzy_face\picture_2242_.png
D:\newfolder43_2\fuzzy_face\picture_2246_.png
D:\newfolder43_2\fuzzy_face\picture_2248_.png
D:\newfolder43_2\fuzzy_face\picture_224_.png
D:\newfolder43_2\fuzzy_face\picture_2252_.png
D:\newfolder43_2\fuzzy_face\picture_2256_.png
D:\newfolder43_2\fuzzy_face\picture_225_.png
D:\newfolder43_2\fuzzy_face\picture_2262_.png
D:\newfolder43_2\fuzzy_face\picture_2268_.png
D:\newfolder43_2\fuzzy_face\picture_226_.png
D:\newfolder43_2\fuzzy_face\picture_2273

D:\newfolder43_2\fuzzy_face\picture_2529_.png
D:\newfolder43_2\fuzzy_face\picture_252_.png
D:\newfolder43_2\fuzzy_face\picture_2530_.png
D:\newfolder43_2\fuzzy_face\picture_2531_.png
D:\newfolder43_2\fuzzy_face\picture_2532_.png
D:\newfolder43_2\fuzzy_face\picture_2533_.png
D:\newfolder43_2\fuzzy_face\picture_2534_.png
D:\newfolder43_2\fuzzy_face\picture_2535_.png
D:\newfolder43_2\fuzzy_face\picture_2536_.png
D:\newfolder43_2\fuzzy_face\picture_2537_.png
D:\newfolder43_2\fuzzy_face\picture_2538_.png
D:\newfolder43_2\fuzzy_face\picture_2539_.png
D:\newfolder43_2\fuzzy_face\picture_253_.png
D:\newfolder43_2\fuzzy_face\picture_2540_.png
D:\newfolder43_2\fuzzy_face\picture_2541_.png
D:\newfolder43_2\fuzzy_face\picture_2542_.png
D:\newfolder43_2\fuzzy_face\picture_2543_.png
D:\newfolder43_2\fuzzy_face\picture_2544_.png
D:\newfolder43_2\fuzzy_face\picture_2545_.png
D:\newfolder43_2\fuzzy_face\picture_2546_.png
D:\newfolder43_2\fuzzy_face\picture_2547_.png
D:\newfolder43_2\fuzzy_face\picture_

D:\newfolder43_2\fuzzy_face\picture_3047_.png
D:\newfolder43_2\fuzzy_face\picture_304_.png
D:\newfolder43_2\fuzzy_face\picture_305_.png
D:\newfolder43_2\fuzzy_face\picture_306_.png
D:\newfolder43_2\fuzzy_face\picture_307_.png
D:\newfolder43_2\fuzzy_face\picture_308_.png
D:\newfolder43_2\fuzzy_face\picture_3091_.png
D:\newfolder43_2\fuzzy_face\picture_3093_.png
D:\newfolder43_2\fuzzy_face\picture_3095_.png
D:\newfolder43_2\fuzzy_face\picture_3099_.png
D:\newfolder43_2\fuzzy_face\picture_309_.png
D:\newfolder43_2\fuzzy_face\picture_30_.png
D:\newfolder43_2\fuzzy_face\picture_3101_.png
D:\newfolder43_2\fuzzy_face\picture_3103_.png
D:\newfolder43_2\fuzzy_face\picture_3107_.png
D:\newfolder43_2\fuzzy_face\picture_310_.png
D:\newfolder43_2\fuzzy_face\picture_311_.png
D:\newfolder43_2\fuzzy_face\picture_3128_.png
D:\newfolder43_2\fuzzy_face\picture_312_.png
D:\newfolder43_2\fuzzy_face\picture_313_.png
D:\newfolder43_2\fuzzy_face\picture_314_.png
D:\newfolder43_2\fuzzy_face\picture_315_.png
D:

D:\newfolder43_2\fuzzy_face\picture_438_.png
D:\newfolder43_2\fuzzy_face\picture_43_.png
D:\newfolder43_2\fuzzy_face\picture_443_.png
D:\newfolder43_2\fuzzy_face\picture_444_.png
D:\newfolder43_2\fuzzy_face\picture_44_.png
D:\newfolder43_2\fuzzy_face\picture_453_.png
D:\newfolder43_2\fuzzy_face\picture_454_.png
D:\newfolder43_2\fuzzy_face\picture_45_.png
D:\newfolder43_2\fuzzy_face\picture_46_.png
D:\newfolder43_2\fuzzy_face\picture_471_.png
D:\newfolder43_2\fuzzy_face\picture_472_.png
D:\newfolder43_2\fuzzy_face\picture_486_.png
D:\newfolder43_2\fuzzy_face\picture_488_.png
D:\newfolder43_2\fuzzy_face\picture_495_.png
D:\newfolder43_2\fuzzy_face\picture_497_.png
D:\newfolder43_2\fuzzy_face\picture_498_.png
D:\newfolder43_2\fuzzy_face\picture_49_.png
D:\newfolder43_2\fuzzy_face\picture_502_.png
D:\newfolder43_2\fuzzy_face\picture_503_.png
D:\newfolder43_2\fuzzy_face\picture_504_.png
D:\newfolder43_2\fuzzy_face\picture_505_.png
D:\newfolder43_2\fuzzy_face\picture_506_.png
D:\newfolder43_

D:\newfolder43_2\fuzzy_face\picture_758_.png
D:\newfolder43_2\fuzzy_face\picture_759_.png
D:\newfolder43_2\fuzzy_face\picture_75_.png
D:\newfolder43_2\fuzzy_face\picture_760_.png
D:\newfolder43_2\fuzzy_face\picture_761_.png
D:\newfolder43_2\fuzzy_face\picture_762_.png
D:\newfolder43_2\fuzzy_face\picture_763_.png
D:\newfolder43_2\fuzzy_face\picture_764_.png
D:\newfolder43_2\fuzzy_face\picture_765_.png
D:\newfolder43_2\fuzzy_face\picture_766_.png
D:\newfolder43_2\fuzzy_face\picture_767_.png
D:\newfolder43_2\fuzzy_face\picture_768_.png
D:\newfolder43_2\fuzzy_face\picture_769_.png
D:\newfolder43_2\fuzzy_face\picture_770_.png
D:\newfolder43_2\fuzzy_face\picture_771_.png
D:\newfolder43_2\fuzzy_face\picture_772_.png
D:\newfolder43_2\fuzzy_face\picture_773_.png
D:\newfolder43_2\fuzzy_face\picture_774_.png
D:\newfolder43_2\fuzzy_face\picture_775_.png
D:\newfolder43_2\fuzzy_face\picture_776_.png
D:\newfolder43_2\fuzzy_face\picture_777_.png
D:\newfolder43_2\fuzzy_face\picture_779_.png
D:\newfolde

In [15]:
#这段测试代码，仍然可以用在txt为训练集的网络中，因为使用var_dataset()返回的也是224个数据，恰好和从txt文件中读回来的一样。
from PIL import Image
import shutil

model.cuda()
model.eval()

test_right_num = 0
test_right_num = t.Tensor([test_right_num]).int()
test_right_num = Variable(test_right_num)

test_dataset1 = txt_dataset("D:\\newfolder43_2_txt")
#i =0
for img0, label0 in test_dataset1:
    img0 = Variable(img0.float()).view(1,1792).cuda()
    label0 = Variable(t.Tensor([label0]).long()).cuda()
    output = model(img0)
    print(output)
    #i=i+1
    #print(i)
    _,pred = t.max(output.cpu(),1)
    #print(pred)
    #print(label0)
    num_correct = (pred==label0.cpu()).sum()
    num_correct = num_correct.int()
    #print()
    test_right_num = test_right_num + num_correct
    #print(test_right_num)
    
acc = (float(test_right_num))/len(test_dataset1)
print(test_right_num)
print(len(test_dataset1))
print(acc)    
    

Variable containing:
-2.2227  2.2124
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-4.2440  4.2152
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-6.9606  6.8886
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-5.2474  5.2507
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-0.5677  0.5962
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-1.6240  1.6696
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-4.5762  4.6006
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-2.9038  2.8792
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-4.9176  5.0134
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-2.6809  2.5911
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-5.3936  5.2998
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-4.6762  4.7996
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Vari

Variable containing:
-4.7568  4.7712
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-5.9236  6.2685
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-5.6574  5.6357
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-5.2139  5.2256
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-6.5536  6.7664
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-6.2232  6.3500
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 1.6192 -1.4660
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-8.6569  8.8103
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-5.7158  5.9261
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-7.2333  7.3414
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-7.6303  8.0310
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-7.6069  7.5349
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Vari

Variable containing:
-5.4547  5.7513
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-5.5274  5.8555
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-6.1357  6.0954
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-5.4817  5.7256
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-7.2828  7.4741
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-5.2852  5.6690
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-4.6250  4.9836
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-5.6328  5.9789
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-7.8458  7.7019
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-4.8902  5.1321
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-9.5171  9.4002
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-10.0263   9.8923
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Va

Variable containing:
-0.2009  0.3442
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-1.2842  1.3983
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-0.4409  0.5743
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-5.8255  5.9857
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-4.2365  4.4362
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 0.2640 -0.1212
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 0.7702 -0.6129
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 1.5753 -1.4998
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 0.7467 -0.6080
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 0.9649 -0.8185
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 0.0107  0.1056
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 3.6334 -3.5598
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Vari

Variable containing:
-11.1174  10.9240
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-8.9824  8.9535
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-9.0308  9.0089
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-8.1552  8.1068
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-7.5831  7.5820
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-10.2892  10.2448
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-7.4592  7.4170
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-9.1133  9.0838
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-9.5063  9.4630
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-8.1213  8.1034
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-9.6665  9.7038
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-8.3466  8.3315
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]



...

Variable containing:
-1.9810  2.0882
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 3.2705 -3.1815
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 3.1401 -3.0535
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 3.2479 -3.0327
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 6.1331 -6.0290
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.8953 -5.6801
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 4.9717 -4.7509
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-1.2774  1.3664
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 4.2141 -4.0171
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 4.3430 -4.1176
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 1.0813 -0.9201
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 4.1556 -3.9797
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Vari

Variable containing:
 7.9231 -7.7112
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.7651 -5.6462
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 4.7723 -4.5811
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.7274 -5.6130
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 8.7665 -8.6175
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 8.3069 -8.1158
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 9.0989 -8.9530
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-1.5425  1.5864
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 7.5934 -7.5067
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 2.9274 -2.7730
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 1.4766 -1.4083
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 2.2893 -2.0944
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Vari

Variable containing:
 3.7798 -3.5652
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 6.7056 -6.5833
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 2.1923 -2.0319
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 3.4008 -3.2891
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 4.9072 -4.7271
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 2.3311 -2.1851
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 3.4185 -3.3488
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 1.8445 -1.6729
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-2.0995  2.0171
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.1255 -4.9236
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 4.8945 -4.7094
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.2619 -5.0497
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Vari

Variable containing:
 5.5367 -5.3514
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 11.2196 -11.0948
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 2.4909 -2.2020
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.1477 -4.9632
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.9286 -5.7741
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 6.6506 -6.5532
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.8052 -5.6284
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 8.3654 -8.2408
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 7.8268 -7.7818
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 4.7687 -4.6085
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.1821 -5.0498
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 11.1524 -11.0525
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]



Variable containing:
 5.0369 -4.8250
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.2348 -5.0175
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 7.0465 -6.8502
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 4.6542 -4.4461
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
-1.1636  1.3771
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.2997 -5.1517
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.3077 -5.0850
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.9848 -5.7909
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 7.1160 -6.9789
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 1.3441 -1.2005
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.1554 -4.9620
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Variable containing:
 5.8364 -5.6969
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

Vari